In [25]:
from hyphenate import hyphenate_word
from model import sp_syllabler
import pickle
from nltk.metrics.distance import edit_distance

In [22]:
# 1
# setting hot func

def convert_to_hot(syl_word):
    hot = []
    i = 0
    while  i < len(syl_word):
        if i == len(syl_word) - 1:
            hot += [1]
            return hot
        if syl_word[i+1] == '-':
            hot += [2]
            i += 2
        else:
            hot += [1]
            i += 1
    return hot

In [12]:
# 1.5
# brier func
def calc_brier(attempted, probability):
    total = 0
    sum_brier = 0
    for word in attempted:
        for c in attempted:
            total += 1
            if c == 2:
                sum_brier += (probability - 1)**2
            elif c == 1:
                sum_brier += (probability - 0)**2
    return (1./total)*(sum_brier)

In [20]:
# 1.8

def inconsistency_grab(attempts, reals, filename = 'cross_compare/incorrect_syls.txt'):
    global converted_back_to_eng
    sum_lev = 0
    incorrect_counter = 0
    incorrect_syl_count = 0
    file = open(filename, 'w+', encoding='utf-8')
    file.write('Attempt' + '\t' + 'Real' + '\n')
    for i in range(0, len(attempts)):
        if attempts[i] != reals[i]:
            incorrect_counter += 1
            a = shitter.insert_syl(converted_back_to_eng[i], attempts[i])
            r = shitter.insert_syl(converted_back_to_eng[i], reals[i])
            if len(a.split('-')) != len(r.split('-')):
                incorrect_syl_count += 1
            sum_lev += edit_distance(a,r,substitution_cost=1, transpositions=True)
            file.write(a + '\t' + r + '\n')
    file.write("Words with errors: %i"%incorrect_counter +'\n')
    file.write("Words with incorrect number of syllables: %i"%incorrect_syl_count +'\n')
    file.write("Total evluated: %i"%len(attempts) +'\n')
    file.write("Perfect accuracy: %.2f"%((len(attempts) - incorrect_counter)/len(attempts)) +'\n')
    file.write("Number of syllables accuracy: %.2f"%((len(attempts) - incorrect_syl_count)/len(attempts)) +'\n')
    file.write("Average Levenshtein Distance(across incorrect words): %.2f"%(sum_lev/incorrect_counter) +'\n')
    file.close()

In [13]:
# 2

def calc_f1(attempted, true, filename = 'cross_compare/evaluation_output_single_pen.txt'):
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    correct_num_char = 0
    total_checked = 0
    evaluation_file = open(filename, 'w+')
    for i in range(0, len(attempted)):
        total_checked += 1
        if (len(attempted[i]) == len(true[i])):
            correct_num_char += 1
            for j in range(0, len(attempted[i])):
                if(attempted[i][j] == true[i][j]):
                    if true[i][j] == 1:
                        true_neg += 1
                    elif true[i][j] == 2:
                        true_pos += 1
                else:
                    if true[i][j] == 1:
                        false_pos += 1
                    elif true[i][j] == 2:
                        false_neg += 1
    evaluation_file.write("Total checked: " + str(total_checked) + '\n')
    evaluation_file.write("Num correctly evaluated character count: " +  str(correct_num_char) + '\n')
    evaluation_file.write("True positives: " + str(true_pos) + '\n')
    evaluation_file.write("True negatives: " + str(true_neg) + '\n')
    evaluation_file.write("False positives: " + str(false_pos) + '\n')
    evaluation_file.write("False negatives: " + str(false_neg) + '\n')
    brier = calc_brier(attempted, probability=0.2)
    precision = true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos + false_neg)
    f_one = 2/((1/precision)+(1/recall))
    evaluation_file.write("precision: " + str(precision) + '\n')
    evaluation_file.write("recall: " + str(recall) +  '\n')
    evaluation_file.write("F1 scores: " + str(f_one) + '\n')
    evaluation_file.write("Brier score: " + str(brier) + '\n')
    evaluation_file.close()

In [23]:
# 3
# grabbing validation data and converting back to English

with open('data/ox/e2i_ortho.pkl', 'rb') as file:
    e2i_ortho = pickle.load(file)

with open('data/ox/x_val_ortho.pkl', 'rb') as file:
    x_val_ortho = pickle.load(file)

with open('data/ox/y_val.pkl', 'rb') as file:
    y_val = pickle.load(file)

converted_back_to_eng = []
for x in x_val_ortho:
    real_word = ""
    for i in x:
            if i != 0:
                real_word += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    converted_back_to_eng += [real_word]    
    
shitter = sp_syllabler(e2i_ortho= e2i_ortho, ortho_input_size=20,latent_dim=32,embed_dim=32 ,max_feat=36)

In [15]:
# 4
# syllabification using liang's algorithm
# one-hot encode results

liang_attempts = []
for word in converted_back_to_eng:
    liang_attempts += ['-'.join(hyphenate_word(word))]

liang_attempts_hot_encoded = [convert_to_hot(word) for word in liang_attempts]

In [26]:
# 5
# f1 scoring liang
# incorrect syl grabbing liang
# omitting padding in reals

reals = []
for x in y_val:
    reals += [[i for i in x if i !=0]]
calc_f1(liang_attempts_hot_encoded, reals, filename='cross_compare/liang_f1.txt')
inconsistency_grab(liang_attempts_hot_encoded, reals)

In [25]:
nums_of_2 = 0
for word in y_val:
    for c in word:
        if c == 2:
            nums_of_2 += 1
print(nums_of_2)

nums_of_2_liang = 0
for word in liang_attempts_hot_encoded:
    for c in word:
        if c == 2:
            nums_of_2_liang += 1
print(nums_of_2_liang)

10706
8955


In [10]:
length_mismatches = []
mismatch_indexes = []

for i in range(0, len(liang_attempts_hot_encoded)):
    if len(liang_attempts_hot_encoded[i]) != len(reals[i]):
        length_mismatches += [(len(liang_attempts_hot_encoded[i]), len(reals[i]))]
        mismatch_indexes += [i]
print(len(mismatch_indexes))

6


['cal', 'iber']
